In [1]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot, show_result
import mmcv
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os
from threading import Thread
import time

In [2]:
config_file = 'DetectoRS/configs/DetectoRS/DetectoRS_mstrain_400_1200_x101_32x4d_40e.py'
# download the checkpoint from model zoo and put it in `checkpoints/`
checkpoint_file = 'DetectoRS/checkpoints/DetectoRS_X101-ed983634.pth'
# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

In [3]:
def render_detection_image(image_file, result):
    image = cv2.imread(image_file)
    h, w, _ = image.shape
    blanks = np.zeros((h, w, 80), np.uint8)
    
    for i in range(80):
        for bbox in result[i]:
            tl = (int(bbox[0]), int(bbox[1]))
            br = (int(bbox[2]), int(bbox[3]))

            blanks[tl[1]:br[1], tl[0]:br[0], i] = int(bbox[-1] * 255)
    
    return blanks

In [4]:
def get_detection_image(img):
    result = inference_detector(model, img)
    return render_detection_image(img, np.array(result[0]))

In [8]:
image_folder = './shopee-product-detection-dataset/train/val/'

In [6]:
def process_image_folder(class_path):
    print('Processing class ' + class_path.split('/')[-2])
    for image_file in os.listdir(class_path):
        if '.jpg' not in image_file:
            continue
        image_path = class_path + image_file
        detection_image = get_detection_image(image_path)
        np.savez_compressed(image_path.replace('.jpg', ''), data = detection_image)
    print('Done ' + class_path.split('/')[-2])    

In [ ]:
threads = []
thread_limit = 5
for class_folder in os.listdir(image_folder):
    class_path = image_folder + class_folder + '/'
    process_image_folder(class_path)
#     threads.append(Thread(target = process_image_folder, args = [class_path]))
#     threads[-1].start()
#     if len(threads) == thread_limit:
#         while True:
#             for thread in reversed(threads):
#                 if not thread.isAlive():
#                     threads.remove(thread)
#                     break
#                 else:
#                     time.sleep(1)

Processing class 00
Done 00
Processing class 01
Done 01
Processing class 02
Done 02
Processing class 03
Done 03
Processing class 04
Done 04
Processing class 05
Done 05
Processing class 06
Done 06
Processing class 07
Done 07
Processing class 08
Done 08
Processing class 09
Done 09
Processing class 10
